# ***Introduction: Business Problem***
The goal is to minimize travel for all Los Angeles residents in need of access to a local gym while promoting health across the city. The objective of this project is to determine optimal locations for a new gym or fitness center within Los Angeles. Specifically, the end result is to compile a list of Los Angeles neighborhoods that have no gyms or fitness centers within 3000 meters from their center points.

# **Data**

The following data resources will be used for this project:

The official list of neighborhoods in Los Angeles.

*   Description: Names of each Los Angeles neighborhood are listed. The scope of this project does not extend to the greater Los Angeles area and is confined to the official neighborhood boundaries under Los Angeles.
*   Data Source: geohub.lacity.org

Coordinates for each Los Angeles neighborhood.

*   Description: Latitude and longitude coordinates for the center of each Los Angeles neighborhood. These coordinates will be used to plot circle markers over a map of Los Angeles and to explore each of them.
* Source: geopy.geocoder package

Venue Data in proximity of each Los Angeles neighborhood.

*   Description: Using Foursquare API a search query will be done against the coordinates for each Los Angeles neighborhood and venue data that is within a radius of 3000 meters of each neighborhood will be generated. This venue data will then be filtered to account for only data related to gyms and fitness centers.
*   Data source: Foursquare API





In [3]:
# METHODOLOGY
%%capture
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from bs4 import BeautifulSoup # library to parse HTML and XML documents

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

# Data Gathering
Getting the link to the LA neighborhood data uploaded in a github repository.

In [4]:
url = "https://github.com/SakshamSomani/ssomani3/blob/main/Data_analysis_project/Strategic-New-Fitness-Centers-LA/LA_Neighborhood.csv" 

Passing the URL to a Pandas dataframe.

In [5]:
df_Neighbs = pd.read_html(url)

Pass the table of interest from the url into a Pandas data frame.

In [6]:
df_LA = df_Neighbs[0]

Dropping unnecessary columns.

In [7]:
df_LA.drop(["Unnamed: 0", "OBJECTID"], inplace=True, axis=1) 

Renaming columns

In [8]:
df_LA.rename(columns = {'name':'Neighborhood'}, inplace = True)

In [9]:

df_LA.head()

,Neighborhood
0,Adams-Normandie
1,Arleta
2,Arlington Heights
3,Atwater Village
4,Baldwin Hills/Crenshaw


In [10]:
df_LA.shape

(114, 1)

Geocoding LA Neighborhoods
Defining a function to retrieve each LA neighborhood's coordinates

In [11]:
def get_latlng(LAneighborhood):
    
    # initialize a variable to None
    lat_lng_coords = None
    
    # loop until all neighborhood coordinates are retrieved using geocoder package
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Los Angeles, California'.format(LAneighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

Calling the defined function to get the neighborhood coordinates and then storing the values into a list.

In [12]:
coordList = [ get_latlng(LAneighborhood) for LAneighborhood in df_LA["Neighborhood"].tolist() ]

Creating a temporary dataframe to populate the coordinates.

In [13]:
df_temp = pd.DataFrame(coordList, columns=['Latitude', 'Longitude'])

Merging the temporary dataframe columns with the df_LA.

In [14]:
df_LA['Latitude'] = df_temp['Latitude']
df_LA['Longitude'] = df_temp['Longitude']

In [15]:
df_LA.head()

,Neighborhood,Latitude,Longitude
0,Adams-Normandie,33.901238,-118.299083
1,Arleta,34.249050,-118.433490
2,Arlington Heights,34.039890,-118.325160
3,Atwater Village,34.119700,-118.258870
4,Baldwin Hills/Crenshaw,34.010428,-118.336776


Data Visualization: Mapping Los Angeles
Getting the coordinates of Los Angeles.

In [16]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.
Creating a map of Los Angeles by passing the latitude and longitude values for each neighborhood.

In [17]:
map_LA = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, neighborhood in zip(df_LA['Latitude'], df_LA['Longitude'], df_LA['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_LA)  
    
map_LA

In [18]:
# saving the map as HTML file
map_LA.save('map_LA3.html')

# Exploring Neighborhood Venues With Foursquare API
Defining foursquare credentials and its version in order to access its database.

In [19]:
CLIENT_ID = 'H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU' # my Foursquare ID
CLIENT_SECRET = 'D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A' # my Foursquare Secret
VERSION = '20201105' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU
CLIENT_SECRET:D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A


Your credentails:
CLIENT_ID: H0ZIZZAEGBFDEGUJYC0XZKX4Z2ROHFX53WN0V5SENBVMDBVU
CLIENT_SECRET:D1HOVQ0X0UQV1JHXSZ0TIC0HFODDSELZH2WTWLJ0U5GQ5E1A
Defining a function to get top venues of each neighborhood within a 500 meter radius.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # making the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # returning only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Running the above function to query each neighborhood through the foursquare API while passing the venue data to a new dataframe called losangeles_venues.



In [24]:
%%capture
losangeles_venues = getNearbyVenues(names=df_LA['Neighborhood'],
                                   latitudes=df_LA['Latitude'],
                                   longitudes=df_LA['Longitude']
                                  )


In [25]:
print(losangeles_venues.shape)
losangeles_venues.head()

(9635, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,33.901238,-118.299083,Popeyes Louisiana Kitchen,33.901664,-118.295854,Fried Chicken Joint
1,Adams-Normandie,33.901238,-118.299083,Starbucks,33.901643,-118.300039,Coffee Shop
2,Adams-Normandie,33.901238,-118.299083,Pho Daily,33.893948,-118.308582,Vietnamese Restaurant
3,Adams-Normandie,33.901238,-118.299083,Spoon House,33.889974,-118.304900,Italian Restaurant
4,Adams-Normandie,33.901238,-118.299083,Waba Grill,33.902347,-118.309726,Japanese Restaurant


Checking how many venues were returned for each neighborhood.

In [26]:
losangeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams-Normandie,100,100,100,100,100,100
Arleta,58,58,58,58,58,58
Arlington Heights,100,100,100,100,100,100
Atwater Village,100,100,100,100,100,100
Baldwin Hills/Crenshaw,76,76,76,76,76,76
Bel-Air,62,62,62,62,62,62
Beverly Crest,44,44,44,44,44,44
Beverly Grove,24,24,24,24,24,24
Beverlywood,100,100,100,100,100,100


Checking how many unique categories can be curated from all the returned venues.

In [27]:
print('There are {} uniques categories.'.format(len(losangeles_venues['Venue Category'].unique())))

There are 411 uniques categories.


Creating a one hot encoding technique that turns each unique venue category into a column and resets the index to be by neighborhood.

In [28]:
# one hot encoding
losangeles_onehot = pd.get_dummies(losangeles_venues[['Venue Category']], prefix="", prefix_sep="")

# adding neighborhood column back to dataframe
losangeles_onehot['Neighborhood'] = losangeles_venues['Neighborhood'] 

# defining a list of column names
cols = losangeles_onehot.columns.tolist()
cols

# moving neighborhood column to the first column
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#applying .reindex() function to reorder
losangeles_onehot = losangeles_onehot.reindex(columns= cols)

#checking result
losangeles_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Car Wash,Caribbean Restaurant,Casino,Cave,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dam,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Stor

In [29]:
#viewing the new dataframe size
losangeles_onehot.shape

(9635, 411)

Grouping the rows by neighborhood and taking the mean for the frequency of occurrence for each venue category.

In [31]:
losangeles_grouped = losangeles_onehot.groupby('Neighborhood').mean().reset_index()
losangeles_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Car Wash,Caribbean Restaurant,Casino,Cave,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Bookstore,College Cafeteria,College Classroom,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dam,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Observatory,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Stor

In [32]:
#confirming new size
losangeles_grouped.shape

(114, 411)

Clustering Neighborhoods by Gym & Fitness Center Data
Getting a count of neighborhoods with at least 1 occurance of a gym or fitness center near them.

In [33]:
len(losangeles_grouped[losangeles_grouped["Gym / Fitness Center"] > 0])

51

Creating a new data frame for the LA neighborhood data on Gyms and Fitness Centers.

In [34]:
LA_gyms = losangeles_grouped[["Neighborhood","Gym / Fitness Center"]]


In [35]:
LA_gyms.head()

,Neighborhood,Gym / Fitness Center
0,Adams-Normandie,0.00
1,Arleta,0.00
2,Arlington Heights,0.02
3,Atwater Village,0.02
4,Baldwin Hills/Crenshaw,0.00


Getting the K-Means Clustering technique setup.

In [36]:
# setting number of clusters
kclusters = 5

losangeles_grouped_clustering = LA_gyms.drop('Neighborhood', 1)

# running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(losangeles_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


array([2, 2, 4, 4, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 4, 0, 2, 1, 2, 2,
       1, 4, 1, 2, 4, 2, 2, 2, 1, 2, 1, 1, 3, 1, 2, 2, 2, 2, 2, 1, 2, 1,
       4, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 4, 1, 3, 4, 1, 2, 2,
       2, 4, 2, 2, 2, 3, 2, 1, 2, 1, 1, 2, 1, 2, 1, 4, 2, 2, 3, 4, 2, 0,
       4, 2, 2, 4, 2, 2, 2, 1, 1, 1, 0, 2], dtype=int32)

In [37]:
LA_merged = LA_gyms.copy()
# adding cluster labels column 
LA_merged["Cluster Labels"] = kmeans.labels_

LA_merged.head() # check the last column

,Neighborhood,Gym / Fitness Center,Cluster Labels
0,Adams-Normandie,0.00,2
1,Arleta,0.00,2
2,Arlington Heights,0.02,4
3,Atwater Village,0.02,4
4,Baldwin Hills/Crenshaw,0.00,2


Merging df_LA with LA_merged to add latitude/longitude for each neighborhood.

In [38]:
LA_merged = LA_merged.join(df_LA.set_index("Neighborhood"), on="Neighborhood")

print(LA_merged.shape)
LA_merged.head() # check the last columns

(114, 5)


,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.00,2,33.901238,-118.299083
1,Arleta,0.00,2,34.249050,-118.433490
2,Arlington Heights,0.02,4,34.039890,-118.325160
3,Atwater Village,0.02,4,34.119700,-118.258870
4,Baldwin Hills/Crenshaw,0.00,2,34.010428,-118.336776


Visualizing the resulting clusters.

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
map_clusters.save('map_clusters.html')

# **Results**

## **Cluster 0** 
(Red)

In [41]:
cluster0 = LA_merged.loc[LA_merged['Cluster Labels'] == 0]
cluster0

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
10,Brentwood,0.040404,0,34.06626,-118.47033
17,Chatsworth Reservoir,0.050000,0,34.22750,-118.63148
87,Studio City,0.040000,0,34.14453,-118.39561
98,Venice,0.040000,0,33.98754,-118.47215


In [42]:
cluster0.shape

(4, 5)

Cluster 0 groups together 28 neighborhoods with a low concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 1 occurrence of a fitness center within a 2000 meter radius from their center points.

## **Cluster 1** 
(Purple)

In [43]:
cluster1 = LA_merged.loc[LA_merged['Cluster Labels'] == 1]
cluster1

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
13,Carthay,0.010000,1,34.056120,-118.373410
19,Cheviot Hills,0.010000,1,34.034030,-118.410560
22,Del Rey,0.010000,1,33.990820,-118.420620
24,Eagle Rock,0.010000,1,34.139270,-118.210870
30,Encino,0.010000,1,34.165380,-118.527110
32,Fairfax,0.010000,1,34.076100,-118.361220
33,Florence,0.010526,1,33.974750,-118.245390
35,Gramercy Park,0.010417,1,34.034053,-118.312335
41,Harbor City,0.010417,1,33.797810,-118.302480
43,Harvard Heights,0.010000,1,34.043390,-118.309030


In [44]:
cluster1.shape

(28, 5)

Cluster 1 groups together 68 neighborhoods with no existence of fitness centers in proximity. The venue data for these neighborhoods generated 0 occurrences of a fitness center within a 2000 meter radius from their center points.

## **Cluster 2**
(Blue)

In [45]:
cluster2 = LA_merged.loc[LA_merged['Cluster Labels'] == 2]
cluster2

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
0,Adams-Normandie,0.0,2,33.901238,-118.299083
1,Arleta,0.0,2,34.249050,-118.433490
4,Baldwin Hills/Crenshaw,0.0,2,34.010428,-118.336776
5,Bel-Air,0.0,2,34.083800,-118.434880
6,Beverly Crest,0.0,2,34.096533,-118.403295
7,Beverly Grove,0.0,2,34.095448,-118.426087
8,Beverlywood,0.0,2,34.051380,-118.401330
9,Boyle Heights,0.0,2,34.033900,-118.205350
11,Broadway-Manchester,0.0,2,34.398960,-118.527785
12,Canoga Park,0.0,2,34.202390,-118.601560


In [46]:
cluster2.shape

(63, 5)

Cluster 2 groups together 4 neighborhoods with a mid-high concentration of fitness centers in proximity. The venue data for these neighborhoods generated 3 occurrences of a fitness center within a 2000 meter radius from their center points.

## **Cluster 3**
(Green)

In [47]:
cluster3 = LA_merged.loc[LA_merged['Cluster Labels'] == 3]
cluster3

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
34,Glassell Park,0.024691,3,34.11925,-118.23039
61,Mid-City,0.030000,3,34.04021,-118.34761
71,Palms,0.030303,3,34.01988,-118.42093
84,Sherman Oaks,0.030000,3,34.15106,-118.44434
113,Woodland Hills,0.030000,3,34.16889,-118.61148


In [48]:
cluster3.shape

(5, 5)

Cluster 3 groups together 9 neighborhoods with a low-mid concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 2 occurrences of a fitness center within a 2000 meter radius from their center points.

## **Cluster 4**
(Orange)

In [49]:
cluster4 = LA_merged.loc[LA_merged['Cluster Labels'] == 4]
cluster4

,Neighborhood,Gym / Fitness Center,Cluster Labels,Latitude,Longitude
2,Arlington Heights,0.020000,4,34.039890,-118.32516
3,Atwater Village,0.020000,4,34.119700,-118.25887
16,Chatsworth,0.020619,4,34.257250,-118.59101
23,Downtown,0.020000,4,34.048340,-118.25564
26,Echo Park,0.020000,4,34.078080,-118.26066
44,Harvard Park,0.020000,4,34.144400,-118.25297
59,Manchester Square,0.019231,4,33.960010,-118.29992
62,Mid-Wilshire,0.020000,4,34.040051,-118.32582
67,North Hollywood,0.020000,4,34.169820,-118.37899
81,Sawtelle,0.020000,4,34.034610,-118.44817


In [50]:
cluster4.shape

(14, 5)

Cluster 4 groups together 2 neighborhoods with a high concentration of fitness centers in proximity. The venue data for these neighborhoods generated at least 4 occurrences of a fitness center within a 2000 meter radius from their center points.

# **Discussion**
Based on the clustering data over the map of Los Angeles, the majority of neighborhoods under Los Angeles county lack access to nearby fitness centers. Very few neighborhoods harbor a mid to high concentration of local fitness centers present. In fact, clusters 2, 3 and 4, which contain a mid to high concentration of fitness centers, can be combined and they would only total 17 neighborhoods out of the 114. Unsurprisingly, clusters 2, 3, and 4 contain suburban neighborhoods with a higher income population. Judging also by their higher concentration of fitness centers, it is safe to recommend to avoid building new fitness centers in these areas since they already have competition established. Even more so, the intent is to give more local access to neighborhoods that lack fitness centers.

Continuing on, the second largest cluster is cluster 0, which list neighborhoods with at least one occurrence of a fitness center in proximity. Cluster 0 contained 30 neighborhoods out of the 114 in Los Angeles and geographically was concentrated towards Western Los Angeles and North West Los Angeles. Competition for memberships is probably not high in these neighborhoods so they can be considered as a secondary tier of neighborhoods for building more fitness centers around.

Every neighborhood that did not generate an occurence of a fitness center within a 2000 meter radius from their center point was assigned to Cluster 1. Cluster 1 had a total of 67 out of 114 neighborhoods assigned to its cluster. Geographically, the neighborhoods under cluster 1 that are located in the most southern side of Los Angeles have the least availability to access a fitness center in proximity.This discrepancy could be due to the fact high concentration of low income residents reside in the southern side of Los Angeles which in the past might have deterred property developers from establishing fitness centers there.

In Addition, there are also several other notable geographic concentrations from cluster 1 that display a lack of available neighborhoods such as East Los Angeles, Central Los Angeles, and the most northern regions of Los Angeles that hug San Fernando. These city regions from cluster 1 are also high population dense areas so there is definitely a high probability of demand for more convenient fitness centers in these areas.


# **Conclusion**
Despite there being some limitations as far as how much venue data can be queried from Foursquare, enough venue data was gathered where an accurate mean frequency of occurrence for gyms and fitness centers can be obtained for each Los Angeles neighborhood. From these mean frequency of occurrences for gyms and fitness centers, five unique clusters were able to be made and each held notable geographic insights. In Particular the most stand out cluster that can be referred to as the ideal list of neighborhoods to target for building new fitness centers is cluster 1. Cluster 1 revealed that there are at least three big and populous regions in Los Angeles that have zero fitness centers established and they are Southern Los Angeles, East los Angeles, and Central Los Angeles. While these regions have historically housed low-income residents, this factor should not limit property developers from pursuing to build fitness centers in these regions. In fact, property developers should aim to build a market of low-cost membership fitness centers in these regions. Better yet, the city of Los Angeles can serve the health needs of its low-income residents better by building community fitness recreational centers in these neighborhoods as well. Overall, there is a market and need to build more fitness centers for the neighborhoods listed under cluster 1.